In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Movie Recomendation System/ratings_small.csv')
df

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [5]:
# Perform a train-test split
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Check the size of the resulting splits
print(f"Train data size: {train_data.shape}")
print(f"Test data size: {test_data.shape}")

Train data size: (80003, 4)
Test data size: (20001, 4)


In [6]:
# Determine the number of unique users and items
num_users = df['userId'].nunique()
num_items = df['movieId'].nunique()
print("number of unique users:", num_users)
print("number of unique items:", num_items)
# Decide on the number of latent factors
num_factors = 20  # This is the 'k' in your model

# Initialize the user and item latent factor matrices with random values
np.random.seed(42)  # For reproducibility
P = np.random.normal(0, 0.1, (num_users, num_factors))
Q = np.random.normal(0, 0.1, (num_items, num_factors))

# Print the shapes of the matrices to verify
print(f"Shape of P (User Latent Factors): {P.shape}")
print(f"Shape of Q (Item Latent Factors): {Q.shape}")


number of unique users: 671
number of unique items: 9066
Shape of P (User Latent Factors): (671, 20)
Shape of Q (Item Latent Factors): (9066, 20)


In [7]:
print(df.head())
print(f"\nTrain data size: {train_data.shape}")
print(f"Test data size: {test_data.shape}")
print("\nnumber of unique users:", num_users)
print("number of unique items:", num_items)
print(f"\nShape of P (User Latent Factors): {P.shape}")
print(f"Shape of Q (Item Latent Factors): {Q.shape}")


   userId  movieId  rating   timestamp
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179
2       1     1061     3.0  1260759182
3       1     1129     2.0  1260759185
4       1     1172     4.0  1260759205

Train data size: (80003, 4)
Test data size: (20001, 4)

number of unique users: 671
number of unique items: 9066

Shape of P (User Latent Factors): (671, 20)
Shape of Q (Item Latent Factors): (9066, 20)


In [8]:
# Find the minimum user_id in the train and test datasets
min_user_id_train = train_data['userId'].min()
min_user_id_test = test_data['movieId'].min()

print(f"Minimum user_id in train dataset: {min_user_id_train}")
print(f"Minimum user_id in test dataset: {min_user_id_test}")

Minimum user_id in train dataset: 1
Minimum user_id in test dataset: 1


Surprise library

In [9]:
!pip install surprise
import numpy as np
import pandas as pd
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", 500)
from surprise import Reader, SVD, Dataset, accuracy
from surprise.model_selection import GridSearchCV, train_test_split, cross_validate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163004 sha256=2c34c1e45c22366b2f928273b85928eb0bac29490b9dbda96e7180972df7f3fc
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [10]:
user_movie_df = df.pivot_table(index=["userId"],columns=["movieId"],values=["rating"]) # user_movie_df oluşturma
user_movie_df

In [11]:
reader = Reader(rating_scale=(1,5)) # puanların hangi sklada gezdiğini veriyoruz
data = Dataset.load_from_df(df[["userId","movieId","rating"]],reader)

In [ ]:
train_data, test_data = train_test_split(data, test_size=0.2)
svd_model = SVD()
svd_model.fit(train_data)
predictions = svd_model.test(test_data)
accuracy.rmse(predictions)

In [23]:
accuracy.rmse(predictions)

RMSE: 0.8951


0.8950621442794826

In [ ]:
svd_model.predict(uid=10,iid=356,verbose=True)

In [ ]:
param_grid = {"n_epochs":[5,10,15,20,25,100],
              "n_factors": [20,50,100,200],
              "reg_all": [0.01, 0.02, 0.05, 0.1]}
# cv: veriyi üç ayrı bölüme ayır ve her defasında ikisiyle model kur diğer bölümü ile test et
# n_jobs: -1 ise işlemcileri full erforması ile kullanmak
# joblib_verbose: raporlama oluştursun mu anlamında
gs = GridSearchCV(SVD,param_grid,measures=["rmse","mae"],cv=3,n_jobs=-1,joblib_verbose=True)
gs.fit(data)

In [20]:
gs.best_score["rmse"]

0.8806818512412748

In [21]:
gs.best_score["mae"]

0.677069000636373

In [22]:
gs.best_params["rmse"]

{'n_epochs': 100, 'n_factors': 200, 'reg_all': 0.1}